### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import seaborn as sns
import pandas as pd
from scipy import stats
import subjects
from statannotations.Annotator import Annotator

### Plot ripples histogram for a single session

In [ ]:
from neuropy.plotting import plot_epochs
from scipy.stats import binned_statistic

sessions = subjects.sd.ratUday1

fig = Fig(grid=(2, 2))
for sub, sess in enumerate(sessions):
    ax = plt.subplot(fig.gs[sub])
    starts = sess.ripple.starts
    durations = sess.ripple.durations
    bins = np.arange(0, sess.eegfile.duration, 600)
    # hist_rpl = np.histogram(starts,bins=bins)[0]
    hist_rpl = binned_statistic(starts, durations, bins=bins, statistic="sum")[0]
    plot_epochs(ax, sess.paradigm)
    ax.plot(bins[:-1] + 300, hist_rpl / 600, "k")
    ax.set_xlabel("Time")
    ax.set_ylabel("Ripple rate")
    print(sess.ripple.metadata)


### Ripple rate first hour **between** SD and NSD (or only restrict to 1h NREM sleep of NSD)
- One possiblity of faster decay of replay during sleep deprivation could be reduced ripple rate at the begining of POST. This may suggest that in order for cells to replay they need sufficient number/amount of exicitation to last for certain duration of time, so a lower ripple rate could interrupt this requirement.

In [ ]:
rpl_rate = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    w = 2 * 3600 # window size
    rpl_rate_sub = sess.ripple_nomerge.time_slice(post[0], post[0] + w).n_epochs / w
    rpl_rate.append(
        pd.DataFrame({"sub": sess.sub_name, "rpl_rate": [rpl_rate_sub], "grp": sess.tag})
    )

rpl_rate = pd.concat(rpl_rate,ignore_index=True)


In [ ]:
_,ax = plt.subplots()
import seaborn as sns

sns.lineplot(data=rpl_rate,x='grp',y='rpl_rate',hue='sub',marker='o')

### Ripple rate first 5 hour **within** SD and NSD across Zt times


In [ ]:
sessions = subjects.mua_sess()

In [ ]:
ripple_rate_df = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    zt_start = np.array([0, 2.5, 5])
    zt_stop = np.array([2.5, 5, 7.5])
    n_epochs = len(zt_start)
    durations = (zt_stop - zt_start) * 3600

    bins = np.vstack((zt_start, zt_stop)).T.flatten() * 3600 + post[0]
    rpls = sess.ripple.starts

    # --- only include rpls having PBE ------
    rpl_epochs = sess.ripple.flatten()
    pbe = sess.pbe.to_dataframe().peak_time.values
    pbe_bool = np.histogram(pbe, rpl_epochs)[0][::2] > 0
    rpls = rpls[pbe_bool]

    binsz = np.diff(bins).mean()
    hist_rpl = np.histogram(rpls, bins=bins)[0][::2]

    ripple_rate_df.append(
        pd.DataFrame(
            {
                "rate": hist_rpl / durations,
                "Zt": [f"{zt_start[_]}-{zt_stop[_]}" for _ in range(n_epochs)],
                "sub": sub,
                "grp": sess.tag,
            }
        ),
    )

ripple_rate_df = pd.concat(ripple_rate_df, ignore_index=True)
# subjects.GroupData().save(ripple_rate_df,'ripple_rate')


In [ ]:
import seaborn as sns
from neuropy.plotting import Fig
from statannotations.Annotator import Annotator

fig = Fig(grid=(7, 6))

ax = fig.subplot(fig.gs[0])
plot_kw = dict(
    data=ripple_rate_df,
    x="Zt",
    y='rate',
    hue="grp",
    hue_order=['NSD','SD'],
    ax=ax,
    dodge=True,
)

sns.stripplot(
    **plot_kw, palette=subjects.colors_sd(1), edgecolor="k", linewidth=0.3, size=3
)
sns.boxplot(**plot_kw, color="k", width=0.5, **subjects.sns_boxplot_kw)
ax.tick_params("x", labelrotation=30)
ax.legend([], frameon=False)
ax.set_ylabel('Ripple_w_PBE rate')
ax.set_xlabel("")

# stats
# Across groups
orders = ripple_rate_df.Zt.unique()
pairs = [((_, "NSD"), (_, "SD")) for _ in orders]
annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
annotator.configure(test="t-test_welch", **subjects.stat_kw)
annotator.apply_and_annotate()
annotator.reset_configuration()

# Within groups
pairs2 = []
for g in ['NSD','SD']:
    pairs2.extend([((orders[_], g), (orders[_ + 1], g)) for _ in range(2)])
annotator = Annotator(pairs=pairs2, **plot_kw, order=orders)
annotator.configure(test="t-test_paired", **subjects.stat_kw)
annotator.apply_and_annotate()
annotator.reset_configuration()


fig.savefig(subjects.figpath_sd/'Ripple_w_PBE_rate')

### Inter-ripple duration (Ripple ISI)


In [ ]:
sessions = subjects.ripple_sess()

In [ ]:
dur_df = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    zts =np.array([0,4,5]) # np.arange(0, 9, 4)
    t_starts = zts * 3600 + post[0]

    acg_zt = []
    firing_neurons = []
    for i, t in enumerate(t_starts):
        starts = sess.ripple.time_slice(t, t + 3600).starts
        dur_df.append(
            pd.DataFrame(
                {"isi": np.diff(starts), "Zt": f"Zt{zts[i]+1}", "grp": sess.tag}
            ),
        )

dur_df = pd.concat(dur_df,ignore_index=True)

In [ ]:

fig = plotting.Fig(grid=(5, 4))
bins = [np.arange(2, 50, 0.1), np.arange(0.5, 50, 0.1)]
for i, g in enumerate(["SD", "NSD"]):
    df = dur_df[dur_df["grp"] == g]
    ax = plt.subplot(fig.gs[i])
    sns.histplot(
        data=df,
        x="isi",
        hue="Zt",
        ax=ax,
        stat="probability",
        bins=bins[i],
        element="poly",
        common_bins=True,
        common_norm=False,
        fill=False,
        legend=None,
        cumulative=True,
        palette="Set2",
    )
    ax.set_xscale("log")
    # ax.set_yscale('log')
    # ax.grid()


### Total/mean ripple duration from first hour to last hour SD

In [ ]:
print(f"#Sessions {len(sessions)}")
rpl_duration = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    # period = np.arange(9) * 3600 + post[0]
    zts = np.array([0, 2.5, 5])
    period = zts * 3600 + post[0]

    for i, t in enumerate(period):

        rpl_duration.append(
            pd.DataFrame(
                {
                    "Zt": f"{zts[i]}-{zts[i]+2.5}",
                    "rpl_duration": [
                        sess.ripple.time_slice(t, t + 2.5 * 3600).durations.sum()
                    ],
                    "grp": sess.tag,
                    "sub": sub,
                }
            ),
        )

rpl_duration = pd.concat(rpl_duration, ignore_index=True)

subjects.GroupData().save(rpl_duration, "ripple_total_duration")


In [ ]:
import seaborn as sns
from neuropy import plotting

fig = plotting.Fig(grid=(4,3))

colors =[subjects.sd.color(1.5),subjects.nsd.color(1.5)]
n_subs=[8,8]
for i, grp in enumerate(["SD", "NSD"]):
    data = rpl_duration[rpl_duration["grp"] == grp]
    ax = fig.subplot(fig.gs[i])
    sns.lineplot(
        data=data,
        x="Zt",
        y="rpl_duration",
        hue="sub",
        marker="o",
        palette=[colors[i]]* n_subs[i],
        ax=ax,
        legend=None,
        ci=None,
    )
    ax.legend('')
    # ax.set_xlim([-1,4])
    # ax.set_ylim([30,450])


# g=sns.relplot(
#     data=rpl_duration,
#     x="epoch",
#     y="rpl_duration",
#     hue="sub",
#     col="grp",
#     # row='rpl_duration',
#     kind="line",
#     marker="o",
#     palette=['gray']*len(sessions),
# )

# [ax.set_xlim([-1,2]) for ax in g.axes.reshape(-1)]
# figure.savefig(subjects.figpath_sd/'ripple_total_duration_zt1_vs_zt5')

### Ripple duration comparison between SD and NSD animals
- I compared the ripple duration distribution for Zt1 and Zt5 between NSD and SD sessions. Ripple durations were pooled across sessions. The distribution curves looked different in Zt1 (seems to be driven by outliers) but exactly the same during Zt5.

In [ ]:
sessions = subjects.ripple_sess()

In [ ]:
rpl_dur, long_rpl = [], []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    zts = np.arange(0, 7.5, 2.5)
    t_starts = zts * 3600 + post[0]

    for ind, t in enumerate(t_starts):
        ep_durations = sess.ripple.time_slice(t, t + 2.5 * 3600).durations
        ep_long_prob =np.count_nonzero(ep_durations>0.1)/len(ep_durations)
        kw = dict(zt=f"{zts[ind]}-{zts[ind]+2.5}", session=sub, grp=sess.tag)
        df1 = pd.DataFrame({"durations": ep_durations, **kw})
        df2 = pd.DataFrame({'prob':[ep_long_prob],**kw})
        rpl_dur.append(df1)
        long_rpl.append(df2)

rpl_dur = pd.concat(rpl_dur, ignore_index=True)
long_rpl= pd.concat(long_rpl, ignore_index=True)


In [ ]:
from stats_funcs import bootstrap

fig = plotting.Fig(grid=(7, 6))

y = ['durations','prob']
for i,data in enumerate([rpl_dur,long_rpl]):
    ax = fig.subplot(fig.gs[i])
    plot_kw = dict(
        data=data,
        x="zt",
        y=y[i],
        hue="grp",
        hue_order=["NSD", "SD"],
        # width=0.5,
        ax=ax,
    )
    sns.boxplot(**plot_kw,**subjects.sns_boxplot_kw,width=0.5)

    if i==1:
        sns.stripplot(**plot_kw,dodge=True,color='k',s=3)
    ax.legend('',frameon=False)
    ax.tick_params('x',rotation=30)
    # ax.set_yscale('log')

    orders = rpl_dur.zt.unique()
    pairs = [((_, "NSD"), (_, "SD")) for _ in orders]
    annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
    annotator.configure(test="t-test_welch", **subjects.stat_kw)
    annotator.apply_and_annotate()
    annotator.reset_configuration()



# bootstrap(pairs,**plot_kw)
fig.savefig(subjects.figpath_sd/'ripple_durations')